# It is compulsory to run it with python  3.11

To make sure tensorflow is correct in Environment change it to python 3.11

In [1]:
# This should be the first block to be executed!
# Install flask to use it later

!pip install flask==2.3.3
!pip install reportlab


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from json import loads, dumps

### 2. Initialize an app/service using Flask
- We use Python Flask to create an app
- We add a base endpoint for the service by mapping the path "/"
- The base URL corresponds to the incoming connection of this notebook (see *7. Start Your App/Service below*)

In [3]:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route("/")
def ehr_app():
    return "This is an Electronic Health Record service."



In [4]:
# Creating POST endpoint to add new patient
# The method expects a JSON object as input
# sample JSON: {"patient_id":1,"patient_fname":"eve","doctor":"Man Steiner","bp_val":95,"chol_val":100}


@app.route('/ArtidisMeasurement/<int:specimen_ID>', methods=['POST'])
def addArtidisDiagnosis(specimen_ID):
    df_artidis = pd.read_csv("artidis_result.csv")
    artidis_record = df_artidis.loc[df_artidis["Specimen_ID"] == specimen_ID]
    df_specimen = pd.read_csv("specimen_database3.csv")


    specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
    specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'ArtidisMeasurement'] = artidis_record["Diagnosis"].values[0]

    df_specimen.update(specimen_row)

    df_specimen.to_csv('specimen_database3.csv', index=False)

    
    specimen_record = df_specimen.loc[df_specimen["Specimen_ID"] == specimen_ID]
    result = specimen_record.to_json(orient="records")
    parsed= loads(result)

    obj = {
        "specimen": parsed
    }
    return obj, 201


In [5]:
@app.route('/addbatch', methods=['POST'])
def addBatch():
    if request.is_json:
        batch_rec = request.get_json()
        df_batch = pd.read_csv("batch_database3.csv")

        # Convert Specimen_ID to int in specimen_rec
        batch_ID = int(batch_rec['Batch_ID'])
        numberSpecimen = int(batch_rec['NumberSpecimen'])


        # Check if Specimen_ID already exists in the CSV file
        if batch_ID in df_batch['Batch_ID'].astype(int).values:
            print("Batch_ID already exists. Not appending.")
            status_code = 400
            obj = {
            "batch": "batch_ID already exists",
            "status_code" : status_code
            }
        
        # return {"error": "Specimen_ID already exists"}, 400  # Return an error response
        else:
            status_code = 201
            # Append the new batch record to the DataFrame
            df_batch_new = df_batch.append(batch_rec, ignore_index=True)
        
            # Write the updated DataFrame to the CSV file
            df_batch_new.to_csv('batch_database3.csv', index=False)
            obj = {
            "batch": batch_rec,
            "status_code" : status_code
            }
            
            



        # Return the response object and a HTTP status code (201 for Created)
        return obj, status_code

In [6]:
#test
json={
"Batch_ID":2,
"NumberSpecimen":4}
def addBatch_test(batch_rec):
    
    df_batch = pd.read_csv("batch_database3.csv")

    # Convert Specimen_ID to int in specimen_rec
    batch_ID = int(batch_rec['Batch_ID'])
    numberSpecimen = int(batch_rec['NumberSpecimen'])


    # Check if Specimen_ID already exists in the CSV file
    if batch_ID in df_batch['Batch_ID'].astype(int).values:
        print("Batch_ID already exists. Not appending.")
        status_code = 400
        obj = {
        "batch": "batch_ID already exists",
        "status_code" : status_code
        }
    
    # return {"error": "Specimen_ID already exists"}, 400  # Return an error response
    else:
        status_code = 201
        # Append the new batch record to the DataFrame
        df_batch_new = df_batch.append(batch_rec, ignore_index=True)
    
        # Write the updated DataFrame to the CSV file
        df_batch_new.to_csv('batch_database3.csv', index=False)
        obj = {
        "batch": batch_rec,
        "status_code" : status_code
        }
            
            



    # Return the response object and a HTTP status code (201 for Created)
    return obj, status_code

#addBatch_test(json)

In [7]:
@app.route('/addspecimen', methods=['POST'])
def addSpecimen():
    if request.is_json:
        specimen_rec = request.get_json()
        df_specimen = pd.read_csv("specimen_database3.csv")

        # Convert Specimen_ID to int in specimen_rec
        specimen_ID = int(specimen_rec['Specimen_ID'])

        # Print statements for debugging
        print("Incoming Specimen_ID:", specimen_rec['Specimen_ID'])
        print("Existing Specimen_IDs:", df_specimen['Specimen_ID'].values)

        # Check if Specimen_ID already exists in the CSV file
        if specimen_ID in df_specimen['Specimen_ID'].astype(int).values:
            print("Specimen_ID already exists. Not appending.")
            status_code = 400
            obj = {
            "specimen": "Specimen_ID already exists",
            "status_code" : status_code
            }
        
        # return {"error": "Specimen_ID already exists"}, 400  # Return an error response
        else:
            status_code = 201
            # Append the new specimen record to the DataFrame
            df_specimen_new = df_specimen.append(specimen_rec, ignore_index=True)
        
            # Write the updated DataFrame to the CSV file
            df_specimen_new.to_csv('specimen_database3.csv', index=False)
            obj = {
            "specimen": specimen_rec,
            "status_code" : status_code
            }
            
            



        # Return the response object and a HTTP status code (201 for Created)
        return obj, status_code

In [8]:
@app.route('/addeligibility2', methods=['POST'])
def addEligibility2():
    if request.is_json:
        specimen_rec = request.get_json()
        df_specimen = pd.read_csv("specimen_database3.csv")
        specimen_ID = int(specimen_rec['Specimen_ID'])
        eligibility_2 = specimen_rec['Eligibility_2']
        print("ELIGIBILITY 2 THAT WAS RECEIVED IS.....")
        print(eligibility_2)
        specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
        specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'Eligibility_2'] = eligibility_2
        df_specimen.update(specimen_row)
        print("Specimen Record is...")
        print(specimen_rec)
        print("Specimen DataBase is...")
        print(df_specimen)
        df_specimen.to_csv('specimen_database3.csv', index=False)

        obj = {
            "specimen": specimen_rec
        }
        return obj, 201
    return {"error": "Request must be JSON"}, 415

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import glob

def load_and_preprocess_data(img):
    data = []

    img = keras.preprocessing.image.load_img(img, target_size=(150,150))
    img_array = keras.preprocessing.image.img_to_array(img)


    data = np.array(img_array)

    return data

@app.route('/classifyImage/<int:specimen_ID>', methods=['POST'])
def classifyImage(specimen_ID):
    pattern = f"Specimen_ID_{specimen_ID}.png"  # Define the pattern
    files = glob.glob(f"*{pattern}*")  # Search for files matching the pattern in the current directory

    image = files[0]
    new_image = load_and_preprocess_data(image)
    
    
    model = load_model('breast_cancer_classifier_model.h5')
    
    result = model.predict(np.expand_dims(new_image, axis=0))
    class_names = ["benign", "malignant"]
    prediction = class_names[int(np.round(result[0]))]
    
    df_specimen = pd.read_csv("specimen_database3.csv")


    specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
    specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'ImageResult'] = prediction 

    df_specimen.update(specimen_row)

    df_specimen.to_csv('specimen_database3.csv', index=False)

    
    specimen_record = df_specimen.loc[df_specimen["Specimen_ID"] == specimen_ID]
    result = specimen_record.to_json(orient="records")
    parsed= loads(result)

    obj = {
        "specimen": parsed
    }
    
    return obj, 201

#obj = classifyImage(5)


2023-12-05 22:12:29.256644: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 22:12:29.265155: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 22:12:29.299375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 22:12:29.299407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 22:12:29.299427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [10]:

@app.route('/histopathologydescription', methods=['POST'])
def histopathologydescription():
    if request.is_json:
        specimen_rec = request.get_json()
        df_specimen = pd.read_csv("specimen_database3.csv")
        specimen_ID = int(specimen_rec['Specimen_ID'])
        histo = specimen_rec['HistopathologyDescription']
        specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
        specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'HistopathologyDescription'] = histo
        df_specimen.update(specimen_row)

        df_specimen.to_csv('specimen_database3.csv', index=False)
        print(specimen_rec)
        obj = {
            "specimen": specimen_rec
        }
        return obj, 201
    return {"error": "Request must be JSON"}, 415

In [11]:
@app.route('/batchcheck/<int:batch_ID>', methods=['GET'])
def batchcheck(batch_ID):
    df_batch = pd.read_csv("batch_database.csv")
    df_specimen = pd.read_csv("specimen_database3.csv")

    batch_record = df_batch.loc[df_batch["Batch_ID"] == batch_ID]
    number_specimen = batch_record.loc[batch_record ['Batch_ID'] == batch_ID, 'NumberOfSpecimen'].values[0]
    count_batch = df_specimen[df_specimen['Batch_ID'] == batch_ID]['Batch_ID'].count()
    print(count_batch)
    print(number_specimen)
    if (count_batch == number_specimen):
        check= True
    else:
        check=False
        
    obj = {
        "check": check 
    }
    return obj, 201

In [12]:
!pip install reportlab
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime
import pandas as pd
import textwrap
import math
def generateReport_test(batch_ID):
    # Create a PDF document
    reportName = f"Report_Batch_{batch_ID}.pdf"
    pdf = canvas.Canvas(reportName, pagesize=letter)

    #Read the data
    df_batch = pd.read_csv("batch_database3.csv")
    df_specimen = pd.read_csv("specimen_database3.csv")

    # Filter the specimens with batch_ID
    batch_specimens = df_specimen[df_specimen['Batch_ID'] == batch_ID]
    batch_record = df_batch.loc[df_batch["Batch_ID"] == batch_ID]
    number_specimen = batch_record.loc[batch_record ['Batch_ID'] == batch_ID, 'NumberSpecimen'].values[0]
    patient_ID = int(batch_specimens.iloc[0]['Patient_ID'])

    #Get the current date for the report
    current_timestamp = datetime.now()
    timestamp = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")


    # Set up the title and content for the report
    title = f"Report for batch ID: {batch_ID}"
    content = [
        f"The report was generated: {timestamp}",
        " ",
        "Batch overview",
        f"Batch ID: {batch_ID}",
        f"Number of specimens in the batch:{number_specimen}", 
        f"Patient_ID: {patient_ID}",
    ]

    # Set up the positioning for title and content
    title_x, title_y = 50, 750
    content_x, content_y = 50, 700

    # Draw title and content on the PDF
    pdf.setFont("Helvetica-Bold", 16)
    pdf.drawString(title_x, title_y, title)

    pdf.setFont("Helvetica", 12)
    for line in content:
        pdf.drawString(content_x, content_y, line)
        content_y -= 20  # Adjust the vertical spacing

    y_position = content_y - 20
    x=70
    print(batch_specimens)
    for index, row in batch_specimens.iterrows():
        # Add parameters to the PDF
        specimen_ID = int(row['Specimen_ID'])
        pdf.drawString(x, y_position, f"Specimen ID: {specimen_ID}")
        y_position -= 20 
        
        if row['Eligibility_1'] == 0:
            row['Eligibility_1'] = False
            pdf.drawString(x, y_position, "Eligibility at the beginning of the process: At the arrival the specimen was not eligible for the analysis.")
            y_position -= 20 
            continue
        else:
            pdf.drawString(x, y_position, f"Eligibility at the beginning of the process: {row['Eligibility_1']}")
            y_position -= 20 
            
        if pd.isna(row['ArtidisMeasurement']):
            pdf.drawString(x, y_position, "Artidis Measurement Result: Artidis measurement could not be performed.")
            y_position -= 20 
        else:
            pdf.drawString(x, y_position, f"Artidis Measurement Result: {row['ArtidisMeasurement']}")
            y_position -= 20 


        if row['Eligibility_2']==0:
            pdf.drawString(x, y_position, "Eligibility after Artidis measurement: After Artidis measurement the specimen got damaged")
            y_position -= 20 
            continue
        elif pd.isna(row['Eligibility_2']):
            print("Eligibility 2 did not have to be assessed")
        else:
            pdf.drawString(x, y_position, "Eligibility after Artidis measurement: Eligible")
            y_position -= 20 
 
        pdf.drawString(x, y_position, f"Histopathology Image Classification Result: {row['ImageResult']}")
        y_position -= 20 
        pdf.drawString(x, y_position, "Histopathology Description:")
        y_position -= 20 
        
        wrapped_text = textwrap.fill(row['HistopathologyDescription'], width=60)
        # Split the wrapped text into lines
        lines = wrapped_text.split('\n')

        # Draw the wrapped text line by line
        for line in lines:
            pdf.drawString(x, y_position, line)
            y_position -= 15

        y_position -= 20 

    print("Report name is..................")
    print(reportName)
    print("Report name type is..................")
    print(type(reportName))
    # Save the PDF file
    pdf.save()
    obj = {
        "reportName": reportName
    }
    return obj, 201

#generateReport_test(987)


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
!pip install reportlab
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime
import pandas as pd

@app.route('/generateReport/<int:batch_ID>', methods=['POST'])
def generateReport(batch_ID):
# Create a PDF document
    reportName = f"Report_Batch_{batch_ID}.pdf"
    pdf = canvas.Canvas(reportName, pagesize=letter)

    #Read the data
    df_batch = pd.read_csv("batch_database3.csv")
    df_specimen = pd.read_csv("specimen_database3.csv")

    # Filter the specimens with batch_ID
    batch_specimens = df_specimen[df_specimen['Batch_ID'] == batch_ID]
    batch_record = df_batch.loc[df_batch["Batch_ID"] == batch_ID]
    number_specimen = batch_record.loc[batch_record ['Batch_ID'] == batch_ID, 'NumberSpecimen'].values[0]
    patient_ID = int(batch_specimens.iloc[0]['Patient_ID'])

    #Get the current date for the report
    current_timestamp = datetime.now()
    timestamp = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")


    # Set up the title and content for the report
    title = f"Report for batch ID: {batch_ID}"
    content = [
        f"The report was generated: {timestamp}",
        " ",
        "Batch overview",
        f"Batch ID: {batch_ID}",
        f"Number of specimens in the batch:{number_specimen}", 
        f"Patient_ID: {patient_ID}",
    ]

    # Set up the positioning for title and content
    title_x, title_y = 50, 750
    content_x, content_y = 50, 700

    # Draw title and content on the PDF
    pdf.setFont("Helvetica-Bold", 16)
    pdf.drawString(title_x, title_y, title)

    pdf.setFont("Helvetica", 12)
    for line in content:
        pdf.drawString(content_x, content_y, line)
        content_y -= 20  # Adjust the vertical spacing

    y_position = content_y - 20
    x=70
    print(batch_specimens)
    for index, row in batch_specimens.iterrows():
        # Add parameters to the PDF
        specimen_ID = int(row['Specimen_ID'])
        pdf.drawString(x, y_position, f"Specimen ID: {specimen_ID}")
        y_position -= 20 
        
        if row['Eligibility_1'] == 0:
            row['Eligibility_1'] = False
            pdf.drawString(x, y_position, "Eligibility at the beginning of the process: At the arrival the specimen was not eligible for the analysis.")
            y_position -= 20 
            continue
        else:
            pdf.drawString(x, y_position, f"Eligibility at the beginning of the process: {row['Eligibility_1']}")
            y_position -= 20 
            
        if pd.isna(row['ArtidisMeasurement']):
            pdf.drawString(x, y_position, "Artidis Measurement Result: Artidis measurement could not be performed.")
            y_position -= 20 
        else:
            pdf.drawString(x, y_position, f"Artidis Measurement Result: {row['ArtidisMeasurement']}")
            y_position -= 20 


        if row['Eligibility_2']==0:
            pdf.drawString(x, y_position, "Eligibility after Artidis measurement: After Artidis measurement the specimen got damaged")
            y_position -= 20 
            continue
        elif pd.isna(row['Eligibility_2']):
            print("Eligibility 2 did not have to be assessed")
        else:
            pdf.drawString(x, y_position, "Eligibility after Artidis measurement: Eligible")
            y_position -= 20 
 
        pdf.drawString(x, y_position, f"Histopathology Image Classification Result: {row['ImageResult']}")
        y_position -= 20 
        pdf.drawString(x, y_position, "Histopathology Description:")
        y_position -= 20 
        
        wrapped_text = textwrap.fill(row['HistopathologyDescription'], width=60)
        # Split the wrapped text into lines
        lines = wrapped_text.split('\n')

        # Draw the wrapped text line by line
        for line in lines:
            pdf.drawString(x, y_position, line)
            y_position -= 15

        y_position -= 20 

    print("Report name is..................")
    print(reportName)
    print("Report name type is..................")
    print(type(reportName))
    # Save the PDF file
    pdf.save()
    obj = {
        "reportName": reportName
    }
    return obj, 201





[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

@app.route('/sendReport', methods=['POST'])
def sendReport():
    if request.is_json:
        # Email details
        record = request.get_json()
        sender_email = 'dummy.dbp.2023@gmail.com'
        sender_password = 'cvdp trca bgjm zfqe'
        receiver_email = 'a.iskrzy.biomed@gmail.com'
        receiver_email = record['Email']
        reportName = record['ReportName']
        # Create message container
        message = MIMEMultipart()
        message['From'] = 'dummy.dbp.2023@gmail.com'
        message['To'] = 'dummy.dbp.2023@gmail.com'
        message['Subject'] = f'New report is available: {reportName}'

        #Content
        content_list = [
            'Hello,',
            'Please find in the attachement a full Report.',
            'Best Regards,',
            'Pathology Department'
            ]
        content = '\n'.join(content_list)
        message.attach(MIMEText(content, 'plain'))
        # Attach the report file
        report_filename = reportName  # Replace with your report file
        attachment = open(report_filename, 'rb')
        report = MIMEBase('application', 'octet-stream')
        report.set_payload((attachment).read())
        encoders.encode_base64(report)
        report.add_header('Content-Disposition', f'attachment; filename= {report_filename}')
        message.attach(report)

        # Establish a secure session with Gmail's outgoing SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, receiver_email, message.as_string())
        server.quit()
        obj = {
            "status": "Report sent successfully"
        }
        
        return obj, 201
    return {"error": "Request must be JSON"}, 415

In [15]:
# the following line of code will make this notebook act like a server
app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.28.188:8080
Press CTRL+C to quit
172.3.8.226 - - [05/Dec/2023 22:13:04] "POST /addbatch HTTP/1.1" 201 -
172.3.27.152 - - [05/Dec/2023 22:13:40] "POST /addspecimen HTTP/1.1" 201 -
Incoming Specimen_ID: 5
Existing Specimen_IDs: []
172.3.40.224 - - [05/Dec/2023 22:13:55] "POST /ArtidisMeasurement/5 HTTP/1.1" 201 -
172.3.40.224 - - [05/Dec/2023 22:14:01] "POST /addeligibility2 HTTP/1.1" 201 -
ELIGIBILITY 2 THAT WAS RECEIVED IS.....
True
Specimen Record is...
{'Specimen_ID': '5', 'Eligibility_2': True}
Specimen DataBase is...
   Specimen_ID  Batch_ID  Patient_ID PatientName   DoctorName  Eligibility_1  \
0            5        34      232134   Max Klaus  Diana Marks           True   

  ArtidisMeasurement Eligibility_2  ImageResult  HistopathologyDescription  
0          malignant          True          NaN                        NaN  
1/1 [=

### 7. Use Postman to test your endpoints
... and have fun! 😃 💃

<img src="image-20230921-121215.png" width="" align="" />

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=69eb62be-3c1b-49e2-ac1f-d13d4e0bc6d3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>